In [ ]:
from scipy.signal import lfilter, lfilter_zi
import numpy as np
import pyaudio as pa
from scipy.io import wavfile

In [ ]:
def tofloat32(x): return (x / np.abs(x).max()).astype(np.float32)

In [ ]:
fs, data = wavfile.read('GOT.wav')
y = tofloat32(data)

In [ ]:
def comb_filter(fs : int, data, tau : float, T60 : float, zi = None) -> tuple[np.array, np.array]:
    N = int(np.round(tau * fs, 0))
    a = np.zeros(N + 1)
    a[[0, N]] = [1, 10 ** (-3 * tau / T60)]
    b = [1.0]
    
    if zi is None: zi = np.zeros(lfilter_zi(b, a).size)
        
    return lfilter(b, a, data, zi = zi)

In [ ]:
p = pa.PyAudio()

chunksize, tau, T60 = 1024, 70e-3, .5

stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
stream.start_stream()

y_chunk = y[ : chunksize]
y_filtered, zi = comb_filter(fs, y_chunk, tau, T60)
y_filtered = tofloat32(y_filtered).tobytes()

i = 1
while len(y_filtered) > 0:
    stream.write(y_filtered)
    y_chunk = y[i * chunksize : (i + 1) * chunksize]
    y_filtered, zi = comb_filter(fs, y_chunk, tau, T60, zi)
    y_filtered = tofloat32(y_filtered).tobytes()
    i += 1
    
stream.stop_stream()
stream.close()
p.terminate()